# Find example
This script is used to find a specific pattern in the output of the SAT solver. If this pattern is found, then the script will print the corresponding example file.

In [85]:
import os
from subprocess import PIPE, Popen
import threading

# Path to the directory containing the sat instances
directorySat = "/mnt/c/Users/robin/OneDrive/University/Library/SAT.CNF/sat/"

# Path to the directory containing the unsat instances
directoryUnSat = "/mnt/c/Users/robin/OneDrive/University/Library/SAT.CNF/unsat/"

# Path to the executable
SAT_exec = "/home/robin/Programs/NapSAT/build/NapSAT"
# SAT_options = [""]
SAT_options = ["", "-wcb", "-lscb", "-rscb"]
additional_options = "-c -sw -cp"

N_THREADS = 12


In [86]:
def search_pattern(filename : str, pattern : str):
    '''
    Runs the SAT solver on the given file and searches for the given pattern in the output.
    If the pattern is found, the filename is printed.
    If some text is printed on stderr, the filename is printed.

    Parameters:
        filename (str): The name of the file to run the SAT solver on
        pattern (str): The pattern to search for in the output

    Returns:
        None
    '''
    for option in SAT_options:
        command = SAT_exec + " " + filename + " " + option + " " + additional_options
        data = os.popen(command)
        output = data.read()
        err = data.close()
        if output.find(pattern) != -1:
            print("\n" + filename + " : " + option + " : Pattern found: " + pattern)
        if err is not None:
            # print the first 10 lines of the error message
            print("\n" + filename + " : " + option + " : " + "ERROR")
            # print("\n".join(output.split("\n")[:10]))

def search_output(root_directory, pattern) :
    """
    Searches for the given pattern in the output of the SAT solver for each file in the given directory.
    If the pattern is found, the filename is printed.
    If some text is printed on stderr, the filename is printed.

    Parameters:
        root_directory (str): The directory to search in
        pattern (str): The pattern to search for in the output

    Returns:
        None
    """
    bench_files = {}

    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if file.endswith(".cnf"):
                if root in bench_files:
                    bench_files[root].append(file)
                else:
                    bench_files[root] = [file]

    dir = list(bench_files.keys())
    dir.sort(key=lambda x: int(x.split("uf")[1].split("-")[0]))

    for directory in dir:
        threads = []
        print ("Directory: " + directory)
        print("Current progress: " + str(0) + "/" + str(len(bench_files[directory])), end="")
        progress = 0
        bench_files[directory].sort()

        for filename in bench_files[directory]:
            print("\rCurrent progress: " + str(progress) + "/" + str(len(bench_files[directory])), end="")
            first = True
            while first or len(threads) == N_THREADS:
                for thread in threads:
                    if not thread.is_alive():
                        threads.remove(thread)
                        progress += 1
                first = False
            thread = threading.Thread(target=search_pattern, args=(directory + "/" + filename, pattern))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
            progress += 1
            print("\rCurrent progress: " + str(progress) + "/" + str(len(bench_files[directory])), end="")
        print()

In [87]:
search_pattern(directorySat + "uf20-91/uf20-0779.cnf", "Lazy")

In [91]:
search_output(directorySat, "FOUND")
search_output(directoryUnSat, "FOUND")

Directory: /mnt/c/Users/robin/OneDrive/University/Library/SAT.CNF/sat/uf20-91
Current progress: 908/1000

KeyboardInterrupt: 